<a href="https://colab.research.google.com/github/flying-bear/infosearch/blob/master/bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Объяснение, почему всё так грустно и медленно

In [2]:
import psutil
env_info = dict(psutil.virtual_memory()._asdict())
for key in env_info:
    if key != 'percent':
        print(key, str(env_info[key] // 1000000000), 'Gb')
    else:
        print(key, str(env_info[key])+'%')

total 13 Gb
available 12 Gb
percent 6.0%
used 0 Gb
free 11 Gb
active 0 Gb
inactive 1 Gb
buffers 0 Gb
cached 1 Gb
shared 0 Gb
slab 0 Gb


## Лекция 2  BM5    

### Imports

In [25]:
import json
import numpy as np
import nltk
import os
import pandas as pd
import re

from google.colab import drive
from math import log
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from time import time

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Constants

In [0]:
k = 2.0
trained_size = 2000
N = trained_size

In [0]:
morph = Mystem()
vectorizer = CountVectorizer()

__important data-independent functions__

In [0]:
def enum_sort(arr): # takes a list and returns a list of ids in the decreasing order of the values from the input
    return [x[0] for x in sorted(enumerate(arr), key=lambda x:x[1], reverse=True)]

In [0]:
def lemmatize(text):
    return [morph.lemmatize(token)[0] for token in nltk.word_tokenize(text)]

### Pre-compute data-dependednt constants

In [17]:
questions = pd.read_csv('gdrive/My Drive/studies/HSE/prog/infosearch/2 bm25/quora_question_pairs_rus.csv', index_col=0).dropna()
questions.head()

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


__only some texts will be used, a part defined by trained_size constant above__

In [0]:
train_texts = questions[:trained_size]['question2'].tolist()
## train_texts = [' '.join(lemmatize(text)) for text in train_texts] ## адово долго!!!

In [0]:
with open('lemmatized.json', 'w') as f:
    f.write(json.dumps(train_texts))

In [0]:
with open('lemmatized.json', 'r') as f:
    train_texts = json.loads(f.read())[:trained_size]

__define mean text length__

In [19]:
lens = [len(text.split()) for text in train_texts]
avgdl = sum(lens)/N
avgdl

9.5675

__precompute a count matrix__
<br> rows - documents
<br> columns - words

In [20]:
X = vectorizer.fit_transform(train_texts)
count_matrix = X.toarray()
count_matrix.shape

(2000, 6632)

__precompute tfs__

In [21]:
tf_matrix = count_matrix / np.array(lens).reshape((-1, 1))
tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

get a vocabulary that has the same indexation as the rows of the count matrix

In [22]:
vocabulary = vectorizer.get_feature_names()
vocabulary[1030:1040]

['вашего',
 'вашей',
 'вашем',
 'вашему',
 'ваши',
 'вашим',
 'ваших',
 'вашу',
 'вблизи',
 'введение']

In [0]:
def preprocess(text, lemm=False):
    if lemm:
        words = lemmatize(text)
    else:
        words = nltk.word_tokenize(text)
    query_modified = list(set(words).intersection(set(vocabulary)))  
    return query_modified

In [27]:
q = 'БЛЯТЬ!111 Я ЗАЕБАЛАСЬ Вороны!22 ебутся в воде !11'
preprocess(q)

['22', '11', 'воде']

__get idfs__

a list of number of docs with a given word for each word

In [28]:
in_n_docs = np.count_nonzero(count_matrix, axis=0)
in_n_docs

array([ 7, 14,  7, ...,  1,  1,  1])

In [0]:
def IDF_modified(word):
    word_id = vocabulary.index(word)
    n = in_n_docs[word_id]
    return log((N - n + 0.5) / (n + 0.5))

In [30]:
IDF_modified('воде')

7.195187320178709

In [31]:
idfs = [IDF_modified(word) for word in vocabulary]
idfs[1000:1010]

[6.6838614462772235,
 7.195187320178709,
 5.893901832250363,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709]

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

### implement tf part of the formula

In [0]:
def modify_tf(tf_value, doc_index, b=0.75):
    l = lens[doc_index]
    return (tf_value * (k + 1.0))/(tf_value + k * (1.0 - b + b * (l/avgdl)))

def modify_tf_matrix(tf_matrix, b=0.75): 
    enumed =  np.ndenumerate(tf_matrix)
    for i, tf_value in enumed:
        doc_index = i[0]
        tf_matrix[i] = modify_tf(tf_value, doc_index, b)
    return tf_matrix*idfs

In [35]:
modified_tf_matrix = modify_tf_matrix(tf_matrix)
modified_tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

__НА СТА ТЫСЯЧАХ НИКАК НЕ МОГУ, У МЕНЯ ОНО НА ДВУХ ТЫСЯЧАХ (trained_size in constants) МИНУТУ КРУТИТСЯ!!11__

### define two bm25 implementations

In [0]:
### реализуйте эту функцию ранжирования векторно
def bm25_vector(query, lemm=False):
    vector = np.array(vectorizer.transform([' '.join(preprocess(query, lemm))]).todense())[0]
    binary_vector = np.vectorize(lambda x: 1.0 if x != 0.0 else 0.0)(vector) ## neutralizes duplictes in the query (non-lineraity)
    return modified_tf_matrix.dot(binary_vector) ## bm25 близость для каждого документа

In [0]:
### реализуйте эту функцию ранжирования итеративно
def bm25_iter(query, lemm=False):
    query_words = preprocess(query, lemm)
    relevance = []
    for i in range(N):
        doc_index = i
        doc_bm25 = 0.0
        for word in set(query_words): ## set neutralizes duplictes in the query
            word_index = vocabulary.index(word)
            tf_value = tf_matrix[(doc_index, word_index)]
            doc_bm25 += idfs[word_index] * modify_tf(tf_value, doc_index)
        relevance.append(doc_bm25)
    return relevance

__Compare performance__

In [0]:
query = 'если честно, мне кажется, что мой итеративный алгоритм работает очень плохо 11 !!'

In [0]:
start = time()
print(len(bm25_vector(query)))
print('TIME non-lemmatized query: ' + str(time() - start))
start = time()
print(len(bm25_vector(query, lemm=True)))
print('TIME lemmatized query: ' + str(time() - start))

2000
TIME non-lemmatized query: 0.027019262313842773
2000
TIME lemmatized query: 40.780805826187134


In [0]:
start = time()
print(len(bm25_iter(query)))
print('TIME non-lemmatized query:', str(time() - start))
start = time()
print(len(bm25_iter(query, lemm=True)))
print('TIME lemmatized query:', str(time() - start))

2000
TIME non-lemmatized query: 3.587538003921509
2000
TIME lemmatized query: 45.14999866485596


__quod erat demonstrandum!__

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу *рождественские каникулы* на нашем корпусе  Quora question pairs. 

__выведу только поиск на первых 2000 (trained_size in constants) документов, должно работать вообще, но моё железо не тянет__<br>
__по *рождественским каникулам* в первых 2000 доков ничего нет, так что на примере другой query__

In [0]:
def search(query, lemm=False, n=N, nonzero=False, vector=True): 
    '''
    searches a given query, returns top n results, by default n = all found (the legth of collection)
    vector flag defines the algorythm (vector is used by default) 
    return format: [(document rank, document id, document text, bm_25), ...]
    '''
    if vector:
        bms = bm25_vector(query, lemm)
    else:
        bms = bm25_iter(query, lemm)
    relevance_sorted_document_ids_top_n = enum_sort(bms)[:n]
    return [(rank, index, np.array(train_texts)[index], bms[index]) for rank, index in enumerate(relevance_sorted_document_ids_top_n)]

In [41]:
query = 'война с водой'
print('query:', query, '\n')
start = time()
response = search(query, n=10)
print('TIME non-lemmatized query:', str(time() - start), '\n')
for rank, document_index, text, bm_25 in response:
    print('relevance rank:', rank+1)
    print('document:', text)
    print('bm_25 =', bm_25, '\n')

query: война с водой 

TIME non-lemmatized query: 0.03602194786071777 

relevance rank: 1
document: мировая война
bm_25 = 9.364436241738273 

relevance rank: 2
document: мировая война iii неизбежна
bm_25 = 5.224180363292414 

relevance rank: 3
document: насколько близка мировая война iii
bm_25 = 3.840680146419764 

relevance rank: 4
document: что такое сирийская гражданская война
bm_25 = 3.840680146419764 

relevance rank: 5
document: можно использовать сенсорные экраны под водой
bm_25 = 3.381176819675889 

relevance rank: 6
document: как работают бассейны с морской водой
bm_25 = 3.381176819675889 

relevance rank: 7
document: 3-я мировая война неизбежна, чем ожидалось
bm_25 = 2.8476281473900436 

relevance rank: 8
document: 3-я мировая война неизбежна, чем ожидалось
bm_25 = 2.8476281473900436 

relevance rank: 9
document: будет ли ядерная война между Индией и Пакистаном
bm_25 = 1.637820508863782 

relevance rank: 10
document: почему важно промыть соленой водой после удаления зубов муд

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

__let us set up the testing__

In [42]:
testable = questions[(questions['is_duplicate'] == 1)&(questions.index < trained_size)][:100]
testable.head()

,question1,question2,is_duplicate
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1
6,как я могу быть хорошим геологом?,"что я должен делать, чтобы быть великим геологом?",1
10,как мне читать и находить комментарии к YouTube,как я могу увидеть все мои комментарии к YouTube,1
11,что может сделать физику легкой для изучения,как вы можете легко научиться физике,1
12,"какой был ваш первый сексуальный опыт, как",какой был ваш первый сексуальный опыт,1


In [43]:
testable.index

Int64Index([  4,   6,  10,  11,  12,  14,  15,  17,  19,  28,  30,  31,  37,
             47,  48,  49,  50,  52,  57,  61,  64,  65,  66,  70,  71,  72,
             73,  78,  83,  84,  85,  87,  91,  92,  94,  99, 103, 106, 112,
            119, 121, 124, 126, 134, 135, 142, 143, 151, 155, 157, 158, 159,
            162, 164, 167, 172, 174, 175, 177, 178, 179, 181, 184, 187, 188,
            189, 190, 192, 193, 196, 197, 198, 199, 202, 208, 209, 214, 215,
            218, 219, 220, 223, 225, 228, 234, 235, 237, 241, 242, 243, 245,
            248, 249, 250, 252, 254, 259, 260, 261, 266],
           dtype='int64')

In [44]:
search(questions.iloc[4]['question1'], n=5)

[(0, 1761, 'что такое свет из', 4.5068478582142895),
 (1, 494, 'что это за картина?', 4.3429893247048135),
 (2, 1432, 'мне все равно, что люди думают обо мне', 4.268822555802342),
 (3, 59, 'это надежные торренты', 3.9793498374648943),
 (4, 1669, 'как это влюбиться', 3.9793498374648943)]

In [0]:
def test_q1_by_id(q1_index):
    top_5_ids = [i for rank, i, text, bm_25 in search(questions.iloc[q1_index]['question1'], n=5)]
    return 1.0 if q1_index in top_5_ids else 0.0

In [46]:
test_q1_by_id(6)

1.0

In [0]:
def test_rank_of_q1_by_id(q1_index):
    top_5_ids = [i for rank, i, text, bm_25 in search(questions.iloc[q1_index]['question1'], n=5)]
    if q1_index in top_5_ids:
        return 1/(top_5_ids.index(q1_index)+1)
    else:
        return 0.0

In [48]:
test_rank_of_q1_by_id(6)

0.2

In [0]:
def test_over_multiple_questions(b, ranked=False, testsize=100): # test for different bs
    testable = questions[(questions['is_duplicate'] == 1)&(questions.index < trained_size)]
    if testsize < len(testable):
        testable = testable[:testsize]
        print('testing on', testsize, 'questions')
    else:
        print('testing on', len(testable), 'questions')
    modified_tf_matrix = modify_tf_matrix(tf_matrix, b=b)
    if ranked:
        test_q1 = test_rank_of_q1_by_id
    else:
        test_q1 = test_q1_by_id
    hit_count = 0.0
    for index in testable.index:
        hit_count += test_q1(index)
    return hit_count/len(testable.index)

__tested on first 100 questions that have a hit__<br>
__can be adjusted by changing *testsize* in test_over_multiple_questions call__

In [50]:
bs = {'BM25': 0.75, 'BM15': 0, 'BM11': 1}
for key in bs:
    b = bs[key]
    print(key, 'b =', b)
    print('boolean precision:', test_over_multiple_questions(b, testsize=100))
    print('rank precision:', test_over_multiple_questions(b, ranked=True, testsize=100))
    print('')

BM25 b = 0.75
testing on 100 questions
boolean precision: 0.51
testing on 100 questions
rank precision: 0.3588333333333333

BM15 b = 0
testing on 100 questions
boolean precision: 0.51
testing on 100 questions
rank precision: 0.3588333333333333

BM11 b = 1
testing on 100 questions
boolean precision: 0.51
testing on 100 questions
rank precision: 0.3588333333333333



In [51]:
bs = {'BM25': 0.75, 'BM15': 0, 'BM11': 1}
for key in bs:
    b = bs[key]
    print(key, 'b =', b)
    print('boolean precision:', test_over_multiple_questions(b, testsize=500))
    print('rank precision:', test_over_multiple_questions(b, ranked=True, testsize=500))
    print('')

BM25 b = 0.75
testing on 500 questions
boolean precision: 0.564
testing on 500 questions
rank precision: 0.4258

BM15 b = 0
testing on 500 questions
boolean precision: 0.564
testing on 500 questions
rank precision: 0.4258

BM11 b = 1
testing on 500 questions
boolean precision: 0.564
testing on 500 questions
rank precision: 0.4258

